In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np
import time

In [2]:
def get_full_data(ids, api, minute):
    """
    ids: 매치 아이디가 담긴 리스트
    api: Riot에서 발급 받은 API
    minute: 분 단위의 값. 설정된 시간 값 이전까지의 매치 데이터를 불러오기 위함 
    """
    start_time = time.time()
    ### 데이터프레임 생성
    use_columns = ['gameId','blueWins', 'blueChamps','blueTotalGolds','blueCurrentGolds','blueTotalLevel'\
                       ,'blueAvgLevel','blueTotalMinionKills','blueTotalJungleMinionKills'
                       ,'blueFirstBlood','blueKill','blueDeath','blueAssist'\
                       ,'blueWardPlaced','blueWardKills','blueFirstTower','blueFirstInhibitor'\
                       ,'blueFirstTowerLane'\
                       ,'blueTowerKills','blueMidTowerKills','blueTopTowerKills','blueBotTowerKills'\
                       ,'blueInhibitor','blueFirstDragon','blueDragonType','blueDragon','blueRiftHeralds'\
                       ,'blueBaron', 'blueFirstBaron'
                       ,'redWins', 'redChamps','redTotalGolds','redCurrentGolds','redTotalLevel'\
                       ,'redAvgLevel','redTotalMinionKills','redTotalJungleMinionKills'
                       ,'redFirstBlood','redKill','redDeath','redAssist'\
                       ,'redWardPlaced','redWardKills','redFirstTower','redFirstInhibitor'\
                       ,'redFirstTowerLane'\
                       ,'redTowerKills','redMidTowerKills','redTopTowerKills','redBotTowerKills'\
                       ,'redInhibitor','redFirstDragon','redDragonType','redDragon','redRiftHeralds'\
                      , 'redBaron', 'redFirstBaron']

    result = pd.DataFrame(columns = use_columns)
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api
    }
    
    for match_id in ids:
        
        try:
            
            ### 타임라인 데이터 크롤링해오기
            URL = "https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/{}".format(match_id)
            res = req.get(URL, headers=headers)

            # API 사용량 제한에 따라서 생기는 오류 처리
            if res.status_code == 200:
                pass

            elif res.status_code == 429:
                while res.status_code != 200:
                    print("API 사용량 초과. 10초 대기")
                    time.sleep(10)
                    res = req.get(URL, headers=headers)

            elif res.status_code == 503:
                while res.status_code != 200:
                    print("알 수 없는 오류")
                    time.sleep(10)
                    res = req.get(URL, headers=headers)

            elif res.status_code == 403:
                while res.status_code != 200:
                    print("API 갱신 필요")
                    break    
                break




            ### 가져올 시간 정의 (~분까지의 경기 데이터)
            timeline = minute*60*1000    # 분을 ms 단위로 변환
            frames = res.json()['frames']

            time_point = 0   # 위치 (인덱스 값): 시작하고 n분까지, 수집하고 싶은 시간까지의 인덱스가 어디있을지 추출
            while True:
                try:
                    timestamps = frames[time_point]['timestamp']
                    # 현재 시점을 기준 시간과 비교
                    if timestamps <= timeline: # n Minute를 설정(Ms단위의 timeline)
                        time_point += 1
                    else:
                        time_point = time_point - 1
                        break
                except:
                    time_point = time_point - 1
                    break



            ### participantFrames 에서 가져올 수 있는 데이터
            participant = res.json()['frames'][time_point]['participantFrames']
            bluetotal_gold, bluecurrent_gold, bluetotal_level, bluetotal_minionkill, bluetotal_jungleminionkill = [],[],[],[],[]
            redtotal_gold, redcurrent_gold, redtotal_level, redtotal_minionkill, redtotal_jungleminionkill = [],[],[],[],[]

            for i in range(len(participant)):
                i = i+1

                # 블루팀의 참가자들
                if 1 <= participant[str(i)]['participantId'] <= 5:
                    bluetotal_gold.append(participant[str(i)]['totalGold'])
                    bluecurrent_gold.append(participant[str(i)]['currentGold'])
                    bluetotal_level.append(participant[str(i)]['level'])
                    bluetotal_minionkill.append(participant[str(i)]['minionsKilled'])
                    bluetotal_jungleminionkill.append(participant[str(i)]['jungleMinionsKilled'])

                # 레드팀의 참가자들
                else:
                    redtotal_gold.append(participant[str(i)]['totalGold'])
                    redcurrent_gold.append(participant[str(i)]['currentGold'])
                    redtotal_level.append(participant[str(i)]['level'])
                    redtotal_minionkill.append(participant[str(i)]['minionsKilled'])
                    redtotal_jungleminionkill.append(participant[str(i)]['jungleMinionsKilled'])




            ### events에서 가져올 수 있는 데이터
            blue_kill, red_kill = 0,0  # 누적 킬 수
            blue_firstkill, red_firstkill = 0,0  # 퍼스트블러드 (0 아니면 1)
            blue_assist, red_assist = 0,0  # 누적 어시스트 수
            red_death, blue_death = 0,0  # 누적 데스 
            blue_wardplace, red_wardplace = 0,0  # 누적 와드 설치 횟수
            blue_wardkill, red_wardkill = 0,0  # 누적 와드 킬 수
            blue_elite, red_elite = 0,0  # 누적 엘리트몬스터 처치 수
            blue_rift, red_rift = 0,0  # RIFTHERALD 처치 수 
            blue_dragon, red_dragon = 0,0  # DRAGON 처치 수
            blue_baron, red_baron = 0,0  #  BARON 처치 수
            blue_firstdragon, red_firstdragon = 0,0  # DRAGON 누가 먼저 먹었는지 (0 아니면 1)
            blue_dragontype, red_dragontype = [],[]  # 처치한 DRAGON 타입
            blue_firstbaron, red_firstbaron = 0,0  # BARON 누가 먼저 먹었는지 (0 아니면 1)
            blue_tower,red_tower = 0,0  # 부신 타워 수
            blue_firsttower, red_firsttower = 0,0  # 첫번째로 누가 타워 부셨는지 (0 아니면 1)
            blue_firsttowerlane, red_firsttowerlane = [],[]  # 첫번째 부신 타워 어떤 라인에 있는지  (먼저 타워를 부신 팀만 값이 있음)
            blue_midtower, red_midtower = 0,0  # 부신 미트 타워 수
            blue_toptower, red_toptower = 0,0  # 부신 탑 타워 수
            blue_bottower, red_bottower = 0,0  # 부신 봇 타워 수
            blue_inhibitor, red_inhibitor = 0,0  # 부신 억제기 수
            blue_firstinhibitor, red_firstinhibitor = 0,0  # 억제기 누가 먼저 부셨는지 (0 아니면 1)  

            frames = res.json()['frames']

            for y in range(1, time_point+1):
                events = frames[y]['events']

                for x in range(len(events)):

                    # 와드 킬
                    if events[x]['type'] == 'WARD_KILL':
                        if 1 <= events[x]['killerId'] <= 5:
                            blue_wardkill += 1
                        else:
                            red_wardkill += 1

                    # 와드 생성
                    elif events[x]['type'] == 'WARD_PLACED':
                        if 1 <= events[x]['creatorId'] <= 5:
                            blue_wardplace += 1
                        else:
                            red_wardplace += 1

                    # 챔피언 킬
                    elif events[x]['type'] == 'CHAMPION_KILL': 
                        if 1 <= events[x]['killerId'] <= 5:
                            # 퍼스트블러드
                            if red_kill ==0 and blue_kill == 0:
                                blue_firstkill += 1
                            blue_kill += 1
                            blue_assist += len(events[x]['assistingParticipantIds'])
                            red_death += 1
                        else:
                            # 퍼스트블러드
                            if red_kill ==0 and blue_kill == 0:
                                red_firstkill += 1
                            red_kill += 1
                            red_assist += len(events[x]['assistingParticipantIds'])
                            blue_death += 1

                    # 엘리트 몬스터 킬
                    elif events[x]['type'] == 'ELITE_MONSTER_KILL':
                        if 1 <= events[x]['killerId'] <= 5:
                            blue_elite += 1
                            # 엘리트 몬스터: DRAGON
                            if events[x]['monsterType']== 'DRAGON':
                                if red_dragon ==0 and blue_dragon == 0:
                                     blue_firstdragon += 1

                                blue_dragontype.append(events[x]['monsterSubType'])
                                blue_dragon += 1
                            # 엘리트 몬스터: RIFTHERALD
                            elif events[x]['monsterType']== 'RIFTHERALD':
                                blue_rift += 1
                            # 엘리트 몬스터: BARON_NASHOR
                            elif events[x]['monsterType']== 'BARON_NASHOR':
                                if red_baron ==0 and blue_dragon == 0:
                                     blue_firstbaron += 1
                                blue_baron += 1
                        else:
                            red_elite += 1
                            # 엘리트 몬스터: DRAGON
                            if events[x]['monsterType']== 'DRAGON':
                                if red_dragon ==0 and blue_dragon == 0:
                                     red_firstdragon += 1
                                red_dragontype.append(events[x]['monsterSubType'])
                                red_dragon += 1
                            # 엘리트 몬스터: RIFTHERALD
                            elif events[x]['monsterType']== 'RIFTHERALD':
                                red_rift += 1
                            # 엘리트 몬스터: BARON_NASHOR
                            elif events[x]['monsterType']== 'BARON_NASHOR':
                                if red_baron ==0 and blue_dragon == 0:
                                     red_firstbaron += 1
                                red_baron += 1

                    # 건물 처치
                    elif events[x]['type'] == 'BUILDING_KILL':
                        if 1 <= events[x]['killerId'] <= 5:
                            # 건물: 타워
                            if events[x]['buildingType'] == 'TOWER_BUILDING':
                                if red_tower == 0 and blue_tower ==0:
                                    blue_firsttower += 1
                                    blue_firsttowerlane.append(events[x]['laneType'])
                                blue_tower += 1
                                # 미드 타워
                                if events[x]['laneType'] == 'MID_LANE':
                                    blue_midtower += 1
                                # 탑 타워
                                elif events[x]['laneType'] == 'TOP_LANE':
                                    blue_toptower += 1
                                # 봇 타워
                                elif events[x]['laneType'] == 'BOT_LANE':
                                    blue_bottower += 1
                            # 건물: 억제기
                            elif events[x]['buildingType'] == 'INHIBITOR_BUILDING':
                                if red_inhibitor == 0 and blue_inhibitor == 0:
                                    blue_firstinhibitor += 1
                                blue_inhibitor += 1
                        else:
                            # 건물: 타워
                            if events[x]['buildingType'] == 'TOWER_BUILDING':
                                if red_tower == 0 and blue_tower ==0:
                                    red_firsttower += 1
                                    red_firsttowerlane.append(events[x]['laneType'])
                                red_tower += 1
                                # 미드 타워
                                if events[x]['laneType'] == 'MID_LANE':
                                    red_midtower += 1
                                # 탑 타워
                                elif events[x]['laneType'] == 'TOP_LANE':
                                    red_toptower += 1
                                # 봇 타워
                                elif events[x]['laneType'] == 'BOT_LANE':
                                    red_bottower += 1
                            # 건물: 억제기
                            elif events[x]['buildingType'] == 'INHIBITOR_BUILDING':
                                if red_inhibitor == 0 and blue_inhibitor ==0:
                                    red_firstinhibitor += 1
                                red_inhibitor += 1          

            ### 여태까지 모은 정보들 종합하기
            data_list = [match_id, 
                         "TBD",
                         "TBD",
                         np.sum(bluetotal_gold),
                         np.sum(bluecurrent_gold),
                         np.sum(bluetotal_level),
                         np.mean(bluetotal_level),
                         np.sum(bluetotal_minionkill),
                         np.sum(bluetotal_jungleminionkill),
                         blue_firstkill,
                         blue_kill,
                         blue_death,
                         blue_assist,
                         blue_wardplace,
                         blue_wardkill,
                         blue_firsttower,
                         blue_firstinhibitor,
                         blue_firsttowerlane,
                         blue_tower,
                         blue_midtower,
                         blue_toptower,
                         blue_bottower,
                         blue_inhibitor,
                         blue_firstdragon,
                         blue_dragontype,
                         blue_dragon,
                         blue_rift,
                         blue_baron,
                         blue_firstbaron,
                         "TBD",
                         "TBD",
                         np.sum(redtotal_gold),
                         np.sum(redcurrent_gold),
                         np.sum(redtotal_level),
                         np.mean(redtotal_level),
                         np.sum(redtotal_minionkill),
                         np.sum(redtotal_jungleminionkill),
                         red_firstkill,
                         red_kill,
                         red_death,
                         red_assist,
                         red_wardplace,
                         red_wardkill,
                         red_firsttower,
                         red_firstinhibitor,
                         red_firsttowerlane,
                         red_tower,
                         red_midtower,
                         red_toptower,
                         red_bottower,
                         red_inhibitor,
                         red_firstdragon,
                         red_dragontype,
                         red_dragon,
                         red_rift,
                         red_baron,
                         red_firstbaron]


            ### 추가 될 데이터프레임 생성
            df_match = pd.DataFrame(np.array([data_list]), columns = use_columns)
            result = result.append(df_match)
            
            
            # 중간 중간 시행횟수 출력
            if len(result)%100 == 0:
                print("현재까지 가져온 매치: {} 개".format(len(result)))
                print("현재까지 소요된 시간: {} 초".format(np.round(time.time()-start_time)))
        
        
        
        # 알 수 없는 에러가 발생했을 때 그냥 무시
        except:
            print("알 수 없는 오류 발생")
        
    print("총 소요된 시간: {} 초".format(np.round(time.time()-start_time)))
    return result

In [4]:
match_id_list = pd.read_csv("match_id.csv")["0"].values
API = "RGAPI-4bf2bb0a-e478-48ef-a2ba-fc43411254af"   # ktshso3

# 데이터 가져오기

In [4]:
# 1차
df1 = get_full_data(ids=match_id_list[:5000], api=API, minute=15)
df1.to_csv("15mins/challenger_15min_1.csv", index=False)

현재까지 가져온 매치: 100 개
현재까지 소요된 시간: 41.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 200 개
현재까지 소요된 시간: 158.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
알 수 없는 오류 발생
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 300 개
현재까지 소요된 시간: 371.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 400 개
현재까지 소요된 시간: 492.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 500 개
현재까지 소요된 시간

API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4000 개
현재까지 소요된 시간: 5055.0 초
알 수 없는 오류 발생
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4100 개
현재까지 소요된 시간: 5176.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4200 개
현재까지 소요된 시간: 5298.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4300 개
현재까지 소요된 시간: 5428.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4400 개
현재까지 소요된 시간: 5549.0 초
API 사용량 초과. 10초 대

In [5]:
# 2차
df2 = get_full_data(ids=match_id_list[5000:10000], api=API, minute=15)
df2.to_csv("15mins/challenger_15min_2.csv", index=False)

API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
알 수 없는 오류 발생
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 100 개
현재까지 소요된 시간: 213.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 200 개
현재까지 소요된 시간: 333.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 300 개
현재까지 소요된 시간: 456.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 400 개
현재까지 소요된 시간: 577.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과

API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4100 개
현재까지 소요된 시간: 5096.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4200 개
현재까지 소요된 시간: 5217.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4300 개
현재까지 소요된 시간: 5341.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4400 개
현재까지 소요된 시간: 5463.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4500 개
현재까지 소요된 시간: 5585.0 초
알 수 없는 오류 발생
알 수 없는 오류 발생
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 

In [5]:
# 3차
match_id_list = pd.read_csv("match_id.csv")["0"].values
df3 = get_full_data(ids=match_id_list[10000:], api=API, minute=15)
df3.to_csv("15mins/challenger_15min_3.csv", index=False)

알 수 없는 오류 발생
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 100 개
현재까지 소요된 시간: 125.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 200 개
현재까지 소요된 시간: 247.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 300 개
현재까지 소요된 시간: 369.0 초
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 400 개
현재까지 소요된 시간: 489.0 초
알 수 없는 오류 발생
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 500 개
현재까지 소요된 시간: 611.0 초
API 사용량 초과. 10

API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
API 사용량 초과. 10초 대기
현재까지 가져온 매치: 4200 개
현재까지 소요된 시간: 5146.0 초
총 소요된 시간: 5155.0 초


In [9]:
df1 = pd.read_csv("15mins/challenger_15min_1.csv")
df2= pd.read_csv("15mins/challenger_15min_2.csv")

In [10]:
data = df1.append(df2).append(df3)
data.to_csv("15mins/challenger_15min.csv", index=False)

In [ ]:
ds